## Overview 

Now that I have my pickled models I can load them in and create some recommendation functions.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from scipy import stats
import scipy as sci

In [2]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from scipy.sparse import csr_matrix
from mlxtend.frequent_patterns import apriori, association_rules

# Function to reduce the memory usage of a DataFrame.
def reduce_memory(df):
    for col in df.columns:
        if df[col].dtype == 'float64':
            df[col] = df[col].astype('float32')
        if df[col].dtype == 'int64':
            df[col] = df[col].astype('int32')
    return df

# Generator function to load data in chunks.
def data_generator(df, chunksize=10000):
    for i in range(0, df.shape[0], chunksize):
        yield df.iloc[i:i+chunksize]


games = reduce_memory(pd.read_csv('../Data/games.csv'))
recommendations = reduce_memory(pd.read_csv('../Data/recommendations.csv'))

In [3]:
# Loading composite rating algorithm

with open('../Models/composite_rating_predictor.pkl', 'rb') as file:
    comp_algo = pickle.load(file)

In [4]:
# Loading is recommended algorithm

with open('../Models/recommender_predictor.pkl', 'rb') as file:
    recommender_algo = pickle.load(file)

In [5]:
from surprise import PredictionImpossible # just in case I hit a user or item that doesn't exist it'll flag impossible 
# instead of returning a random guess 


# this is my composite rating algorithm, the current user and item value included are a random user who put in 900 hours 
# into Rainbow Six Siege (the game listed) yet despite that rated it negatively.  This composite rating of mine takes that 
# into consideration and gave him a much lower predicted rating.
user_id = 987098
item_id = 359550


# SVD appears to return default scores for items and users that do not exist.  This if statement is meant to avoid that.
if user_id not in recommendations['user_id'].unique():
    print(f"Error: User {user_id} does not exist in the training set.")
elif item_id not in recommendations['app_id'].unique():
    print(f"Error: Item {item_id} does not exist in the training set.")
else:
    loaded_pred = comp_algo.predict(user_id, item_id)
    print(f"The estimated rating for user {user_id} and item {item_id} is {loaded_pred.est}")

The estimated rating for user 987098 and item 359550 is 4.745550590913284


In [6]:
# this is the is_recommended algorithm.  Since it also learns implicit features it recognizes that this user has put a ton
# of time into this game and rates it higher despite the user rating it negatively. It brings up the issue of subjectivity,
# did this user not recommend the game to be funny?  Is there something the creators did which altered his opinion?
# Neither results are necessarily incorrect, they both take the information they're presented with just weighted in different
# manners.  

user_id = 987098
item_id = 359550

if user_id not in recommendations['user_id'].unique():
    print(f"Error: User {user_id} does not exist in the training set.")
elif item_id not in recommendations['app_id'].unique():
    print(f"Error: Item {item_id} does not exist in the training set.")
else:
    loaded_pred = recommender_algo.predict(user_id, item_id)
    print(f"The estimated rating for user {user_id} and item {item_id} is {loaded_pred.est}")

The estimated rating for user 987098 and item 359550 is 0.8679839396964601


I wrote above in comments but I'll just summarize here.  Both algorithms learned in different ways depending on the implicit features they absorbed through SVDpp.  The algorithm that relies on the composite rating I generated notices the flags the user put up and thusly penalizes them, agreeing that because the user meets a certain criteria and they don't recommend the game in question that it should be rated lower.  The is recommended algorithm implicitly realizes that even if that user doesn't recommend the game they've put a ton of time into it and thus it gets a rating closer to 1 than 0.  Neither is necessarily incorrect, they simply learned different things.

In [7]:
# one of two functions to get a top 10 list of estimated ratings for a user based on what it predicts the user would like
# this one is for the comp_algo with a scale of 0-10.



# Returns the list of games owned by whatever user_id is keyed into the final function. Shared among both algorithms.
def get_owned_games(user_id, recommendations):
    return recommendations[recommendations['user_id'] == user_id]['app_id'].unique()

# unique per algorithm, gets the predictions of the user that is keyed in to the final function
def predict_comp_ratings(user_id, games_to_predict):
    predictions = [(game, comp_algo.predict(user_id, game).est) for game in games_to_predict]
    return sorted(predictions, key=lambda x: x[1], reverse=True)

# function that combines all the output 
def recommend_comp_games(user_id):
    
    # Check if user_id exists in the training set
    # Unfortunately without this the algorithm will just return a default list for anyone.
    if user_id not in recommendations['user_id'].unique():
        print(f"Error: user {user_id} does not exist in the training set.")
        return []

    # My model appears to have learned a little too well, it has a tendency to recommend the same games to everyone 
    # because it has found that those games perform almost unanimously well. This popularity metric changes up the output 
    # by a fair amount.
    popular_games = games[games['user_reviews'] > 10000]

    # Collects the list of games for the user from the first function.
    owned_games = get_owned_games(user_id, recommendations)

    # Filter out the games already owned by the user and cross-reference it with the total number of reviews the game has
    games_to_predict = popular_games[~popular_games['app_id'].isin(owned_games)]['app_id']

    # Predict ratings for the filtered games
    top_predictions = predict_comp_ratings(user_id, games_to_predict)[:10]

    # Takes the id's of the games returned and maps them to their title for easier legibility, additionally includes 
    # the models predicted rating for the game by the user 
    top_games_with_ratings = [(games[games['app_id'] == game_id]['title'].iloc[0], rating) 
                              for game_id, rating in top_predictions]

    return top_games_with_ratings

In [8]:
# identical function but for the rec_algo which is a 0-1 scale.

def predict_recommended_ratings(user_id, games_to_predict):
    
    # since its a 0-1 scale have to ensure rating returns as a float to ensure a better idea of whether a game is recommended
    # or not.
    predictions = [(game, float(recommender_algo.predict(user_id, game).est)) for game in games_to_predict]
    return sorted(predictions, key=lambda x: x[1], reverse=True)

def recommend_rec_games(user_id):
     
        
    if user_id not in recommendations['user_id'].unique():
        print(f"Error: user {user_id} does not exist in the training set.")
        return []
    
    owned_games = get_owned_games(user_id, recommendations)
    
    # once again the popularity metric penalizes the model for recommending low played but high ranking games to the user
    games_to_predict = games[(~games['app_id'].isin(owned_games)) & (games['user_reviews'] > 10000)]['app_id']
    top_predictions = predict_recommended_ratings(user_id, games_to_predict)[:10]
    
    top_games_with_ratings = [(games[games['app_id'] == game_id]['title'].iloc[0], rating) 
                              for game_id, rating in top_predictions]
    return top_games_with_ratings

In [9]:
recommend_rec_games(0) # calling the 0-1 scale algorithm

[('A Short Hike', 0.9924125343543445),
 ('Aseprite', 0.9883104327435392),
 ('Rhythm Doctor', 0.9880431602954187),
 ('Terraria', 0.9864730291409904),
 ('Hades', 0.9856765131629152),
 ('Vampire Survivors', 0.9847364608411153),
 ('Entropy : Zero 2', 0.9846111366114456),
 ('OneShot', 0.9839081028437245),
 ('Portal 2', 0.9837804406334922),
 ('Finding Paradise', 0.9836854660264128)]

In [10]:
recommend_comp_games(0) # calling the 0-10 scale algorithm

[('Kung Fury', 7.893856197449941),
 ('Finding Paradise', 7.875466930301883),
 ('OneShot', 7.8386029072420085),
 ('Call of Juarez: Gunslinger', 7.824269883694496),
 ('Gorogoa', 7.818253135815099),
 ('Montaro', 7.779746047001589),
 ('Eternal Senia', 7.754772580247042),
 ('A Short Hike', 7.739986974339837),
 ('Quake', 7.738840422621572),
 ('Fran Bow', 7.7072577325659966)]

## Overview

The models are rather tricky, with no threshholds or limits holding them back they unanimously select the top performing games (in its opinion) and recommend them frequently to almost every player regardless of their preferences.  Yet despite that, their estimated guessings of scores for players are fairly consistent and give me results I would expect.  The 0-1 scale recommender seems a bit better at finding unanimously popular games whereas the composite rating algorithm takes some user nuances into account and returns some interesting and unexpected results.

I don't think any of them on their own would perform very well in a real setting and that's because I think the model could be very benefitted by combined models.  Models I can see assisting and refining the output would be a similar users matrix utilizing KNN, tag based filtering (potentially an item similarity matrix using tags and vectorization), and some more intuitive model thresholds.  Popularity was a good start to threshholding but adding more features would improve the model significantly.

Overall the models do the task that they were trained to do.  They just do not understand the complexities between each user and item.  That information can be provided through other means at a later date, for now the ability to understand how a user would rate a certain item is sufficient.  